In [31]:
import numpy as nmp 
import math
import os
import os.path

In [32]:
def minimal_required_index(arg,x_list):
    for i in range(0,len(x_list)):
        if (x_list[i] > arg):
            required_index = i
            break
    return required_index

def is_equidistant(x_list):
    h = round((x_list[1]-x_list[0]),10)
    for i in range(1,len(x_list)):
        if (round((x_list[i]-x_list[i-1]),10) != h):
            return False 
    return True

def is_on_left(arg,x_list):
    if (arg<=((x_list[0]+x_list[len(x_list)-1])/2)):
        return True
    else:
        return False

def first_order_sub(x1,x0,y1,y0):
    return (y1-y0)/(x1-x0)

def second_order_sub(x2,x1,x0,y2,y1,y0):
    return (first_order_sub(x2,x1,y2,y1)-first_order_sub(x1,x0,y1,y0))/(x2-x0)

In [33]:
def linear_interpolation(arg,arr_x,arr_y):
    required_index = minimal_required_index(arg,arr_x)
    c_A = (arr_y[required_index]-arr_y[required_index-1])/(arr_x[required_index]-arr_x[required_index-1])
    c_B = arr_y[required_index-1]-(c_A*arr_x[required_index-1])
    return (c_A*arg + c_B)


def polynomial_interpolation(arg,arr_x,arr_y):
    required_index = minimal_required_index(arg,arr_x)
    coefficients = nmp.array([[arr_x[required_index-2]*arr_x[required_index-2],arr_x[required_index-2],1], 
                       [arr_x[required_index-1]*arr_x[required_index-1],arr_x[required_index-1],1], 
                       [arr_x[required_index]*arr_x[required_index],arr_x[required_index],1]])
    
    results = nmp.array([arr_y[required_index-2],arr_y[required_index-1],arr_y[required_index]])
    ans = nmp.linalg.solve(coefficients, results)
    return (ans[0]*arg*arg+ans[1]*arg+ans[2])

In [34]:
def lagrange_interpolation(arg,arr_x,arr_y):
        res = 0
        for i in range (0,len(arr_y)):
            dividend=1; divisor=1
            for j in range(0,len(arr_x)):
                if i!=j:
                    dividend*=(arg-arr_x[j])
                    divisor*=(arr_x[i]-arr_x[j]) 
            res += (dividend/divisor)*arr_y[i]
        return res

In [35]:
def newton_nonequidistant(arg,arr_x,arr_y,index_1,index_2,index_3):
    first_order = first_order_sub(arr_x[index_2],arr_x[index_1],arr_y[index_2],arr_y[index_1])
    second_order = second_order_sub(arr_x[index_3],arr_x[index_2],arr_x[index_1],arr_y[index_3],arr_y[index_2],arr_y[index_1])
    
    return arr_y[index_1]+(arg-arr_x[index_1])* first_order +(arg-arr_x[index_1])*(arg-arr_x[index_2])*second_order

In [36]:
def newton_first_order(arg,arr_x,arr_y):
    required_index = minimal_required_index(arg,arr_x)
    required_index -= 1
    res = arr_y[required_index]
    iterator = len(arr_x) - required_index - 1
    subs = []
    h = round((arr_x[1]-arr_x[0]),10) 
    t = (arg-arr_x[required_index])/h   
    
    for i in range(iterator):
        subs.append([])
    for i in range(required_index,len(arr_x)-1):
        subs[0].append(arr_y[i+1]-arr_y[i])
    for i in range(1,len(subs)):
        for j in range(len(subs[i-1])-1): 
            subs[i].append(subs[i-1][j+1] - subs[i-1][j])
    for i in range(1,iterator):
        acc = 1
        for j in range(i):
            acc*=(t-j)
            if (j!=0):
                acc/=j
        acc *= subs[i-1][0]
        res += acc
    return res
    
def newton_second_order(arg,arr_x,arr_y):
    required_index = minimal_required_index(arg,arr_x)
    res = arr_y[required_index]
    iterator = required_index
    subs = []
    h = round((arr_x[1]-arr_x[0]),10) 
    t = (arg-arr_x[required_index])/h 
    
    for i in range(iterator):
        subs.append([])    
    for i in range(0,required_index):
        subs[0].append(arr_y[i+1]-arr_y[i])
    for i in range(1,len(subs)):
        for j in range(len(subs[i-1])-1): 
            subs[i].append(subs[i-1][j+1] - subs[i-1][j])
    for i in range(1,iterator-1):
        acc = 1
        for j in range(i):
            acc*=(t+j)
            if (j!=0):
                acc/=j
        acc *= subs[i-1][len(subs[i-1])-i]
        res += acc        
    return res

In [37]:
print("Введите имя файла для считывания\n")

is_correct = True

try:
    while(True):
        testFilename = input()
        if os.path.exists(testFilename):
            break
        else:
            print("Такого файла не существует. Проверьте правильность ввода и попробуйте еще раз.")
        
    input_file = open(testFilename,'r')
    string_x = str(input_file.readline())
    string_y = str(input_file.readline())
    input_file.close()
    
    x_arr = string_x.split()
    y_arr = string_y.split()
    
    x_list = nmp.asarray(list(map(float,x_arr)))
    y_list = nmp.asarray(list(map(float,y_arr)))
    
    element_max = nmp.amax(x_list)
    element_min = nmp.amin(x_list)
    
    if (len(x_list)!=len(y_list)):
        print("Несоответствующее количество Х и Y. Проверьте правильность данных и попробуйте еще раз.")
        is_correct = False
except ValueError as e:
    print("В файле найдены нечисловые данные. Проверьте правильность данных и попробуйте еще раз.")
    is_correct = False
    
if (is_correct):
    print("Успешное чтение данных.\nВведите значение аргумента, используемого для вычисления приближенного значения функции")
    while(True):
        try:
            arg = float(input())
            if (arg > element_max or arg < element_min):
                print("У нас лабораторная по интерполяции, а не экстраполяции. Введите число, входящее в заданные границы.")
            else:
                break
        except ValueError as e:
            print("Ввод должен быть числом. Попробуйте еще раз.")
            
    print("Линейная интерполяция - " + str(linear_interpolation(arg,x_list,y_list)))
    print("Квадратичная интерполяция - " + str(polynomial_interpolation(arg,x_list,y_list)))
    print("Интерполяция с помощью многочлена Лагранжа - " + str(lagrange_interpolation(arg,x_list,y_list)))
    print("Интерполяция с помощью неравноотстоящих узлов (0,1,2) - " + str(newton_nonequidistant(arg,x_list,y_list,0,1,2)))
    print("Интерполяция с помощью неравноотстоящих узлов (1,2,3) - " + str(newton_nonequidistant(arg,x_list,y_list,1,2,3)))
    
    if (is_equidistant(x_list)):
        if (is_on_left(arg,x_list)):
            print("Интерполяция по первой формуле Ньютона (т.к. аргумент в левой половине) - " + str(newton_first_order(arg,x_list,y_list)))
        else:
            print("Интерполяция по второй формуле Ньютона (т.к. аргумент в правой половине) - " + str(newton_second_order(arg,x_list,y_list)))
    else:
        print("Функция не содержит в себе равноотстоящих точек")

Введите имя файла для считывания

lab3input2.txt
Успешное чтение данных.
Введите значение аргумента, используемого для вычисления приближенного значения функции
1.557
Линейная интерполяция - 5.2529499999999985
Квадратичная интерполяция - 5.253268989999999
Интерполяция с помощью многочлена Лагранжа - 5.254844268587183
Интерполяция с помощью неравноотстоящих узлов (0,1,2) - 5.284790799999985
Интерполяция с помощью неравноотстоящих узлов (1,2,3) - 5.236685434999994
Интерполяция по второй формуле Ньютона (т.к. аргумент в правой половине) - 5.253232959103
